In [1]:
"""
REFS:
1) Wildon, Vertices of Specht Modules and Blocks of the Symmetric Group
2) Murphy, The Idempotents of the Symmetric Group and Nakayama's Conjecture
3) Murphy, A new construction of Young's seminormal representation of the symmetric groups
""";

In [2]:
"""
NOTES: eigenvalues appear to have multiplicity 1
""";

In [7]:
SymmetricGroupAlgebra(GF(3),4).dft()

24 x 24 dense matrix over Finite Field of size 3 (use the '.str()' method to see the entries)

In [4]:
SGA_Q3 = SymmetricGroupAlgebra(QQ,3)

In [5]:
SGA_Q3.dft()

[   1    1    1    1    1    1]
[   1  1/2   -1 -1/2 -1/2  1/2]
[   0  3/4    0  3/4 -3/4 -3/4]
[   0    1    0   -1    1   -1]
[   1 -1/2    1 -1/2 -1/2 -1/2]
[   1   -1   -1    1    1   -1]

In [6]:
SGA_F7 = SymmetricGroupAlgebra(GF(7),3)

In [7]:
len(set(SymmetricGroupAlgebra(QQ,4).dft().eigenvalues()))

24

In [8]:
SGA_F7.specht_module(Partition([1,1,1]))

Specht module of [(0, 0), (1, 0), (2, 0)] over Finite Field of size 7

In [8]:
SGA_F3 = SymmetricGroupAlgebra(GF(3),3)

In [10]:
#one cannot perform the DFT when p | n!
try:
    print(SGA_F3.dft())
except ZeroDivisionError:
    print("Modular case not handled!")

[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]


In [11]:
#construct Pierce decomposition of SGA into sum of blocks

In [12]:
#implements modular Fourier transform
#project v onto each block U_i = F_p[S_n]*e_i using \pi_i: v |--> v*e_i as a projection
#this is just a change of basis
def modular_fourier_transform(p,n,nu=1):
    #instantiate group algebra
    SGA_GFp_n = SymmetricGroupAlgebra(GF(p**nu),n)
    #compute the primitive central orthogonal idempotents
    idempotents = SGA_GFp_n.central_orthogonal_idempotents()
    # project v onto each block U_i = F_p[S_n]*e_i via \pi_i: v |--> v*e_i
    B = SGA_GFp_n.basis()
    blocks = [SGA_GFp_n.submodule([b * idem for b in B]) for idem in idempotents]
    # compute the list of basis vectors lifted to the SGA from each block
    block_decomposition_basis = [u.lift() for block in blocks for u in block.basis()]
    # construct the matrix to the standard basis in the order given by the group
    G = SGA_GFp_n.group()
    mat = [[b[g] for b in block_decomposition_basis] for g in G]
    return matrix(SGA_GFp_n.base_ring(), mat)

In [13]:
#compute the splitting field of a polynomial
def splitting_field(f,K,splitting_degs=[]):
    #define polynomial ring we're working over
    R.<x> = PolynomialRing(K)
    #factor f over K into irreducible factors 
    factor_f = f(R.gen()).factor()
    #choose first non-linear factor. just pick last one since they're ordered
    nonlinear_factor_f = list(factor_f)[-1][0]
    splitting_degs.append(nonlinear_factor_f.degree())
    if nonlinear_factor_f.degree() == 1:
        return K, splitting_degs
    #form the quotient ring K_{i+1} = K_i[x]/(f_i(x))
    K_ext = R.quotient(nonlinear_factor_f)
    #repeat process until f is completely factored
    return splitting_field(f,K_ext,splitting_degs)

In [14]:
#compute the splitting degree of the splitting field of a polynomial by computing the lcm
def splitting_field_degree(f):
    return lcm([factor[0].degree()**factor[1] for factor in list(f.factor())])

In [15]:
#A fast way for computing the order of 𝑀 is thus 
#to compute the characteristic polynomial P_L of M
#factor it over F_p and check then if prime-divisors of p^k-1
#(for 𝑘 the degree of an involved irreducible polynomial) 
#divide the order
def order_finite_field(M):
    L = M.nrows()
    p = len(M.base_ring())
    P_L = M.charpoly(); P_L
    char_poly_factored = P_L.factor(); char_poly_factored
    degree_list = [item[0].degree() for item in char_poly_factored]
    U = p^(L-1)*prod(p^k-1 for k in degree_list)
    for div in divisors(U):
        if M^div == matrix.identity(L):
            return div

In [16]:
#compute the order by computeing the eigenvalues of the matrix, which are distinct, and taking the lcm
def order_via_lcm(p,n):
    eigs = modular_fourier_transform(p,n,nu=1).eigenvalues()
    return lcm([eig.multiplicative_order() for eig in eigs])

In [17]:
p=3; n=4

In [18]:
#compute the modular Fourier transform for an example element v
mft = modular_fourier_transform(p,n); mft

24 x 24 dense matrix over Finite Field of size 3 (use the '.str()' method to see the entries)

In [19]:
print(mft.str())

[1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0]
[1 2 2 1 1 0 0 0 0 2 2 2 2 2 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
[2 1 0 0 0 1 0 0 0 2 2 0 0 0 2 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
[0 0 2 0 1 0 1 0 0 0 0 2 0 2 0 2 0 0 0 0 1 0 0 0]
[1 2 2 0 1 0 0 1 0 1 1 1 0 1 0 0 2 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0]
[0 0 2 1 0 0 0 0 1 0 0 2 2 0 0 0 0 2 0 0 1 0 0 0]
[1 0 2 0 0 2 1 0 1 2 0 2 0 0 2 2 0 2 0 0 0 0 0 1]
[1 0 2 0 0 2 0 1 1 1 0 1 0 0 1 0 2 1 0 0 0 0 1 0]
[1 2 1 1 1 2 1 0 1 1 1 2 1 1 1 1 0 1 1 0 0 0 0 0]
[1 2 2 0 1 2 0 1 1 2 2 2 0 2 2 0 1 2 0 1 0 0 0 0]
[1 1 2 2 2 1 2 2 2 2 1 2 1 1 1 1 2 1 0 0 0 0 0 1]
[2 0 1 0 2 1 2 2 2 2 0 2 0 2 2 2 1 2 0 0 0 0 1 0]


In [20]:
#perform Hill cipher with MFT(p,n) as matrix
txt = "hom"
bit_str = flatten([list(bin_str)[2:] for bin_str in [bin(num) for num in list(txt.encode())]])
encoded_vec = [GF(p)(bit) for bit in bit_str] + (factorial(n)-len(bit_str))*[0]
print(len(encoded_vec))
''.join([str(i) for i in mft*vector(encoded_vec)])

24


'222210121111201020111101'

In [21]:
modular_fourier_transform(7,3) == SGA_F7.dft()

False

In [22]:
#note: this matrix has (almost) only has 1's and -1's
M = modular_fourier_transform(3,4,nu=1)
print(M.str())

[1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0]
[1 2 2 1 1 0 0 0 0 2 2 2 2 2 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
[2 1 0 0 0 1 0 0 0 2 2 0 0 0 2 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
[0 0 2 0 1 0 1 0 0 0 0 2 0 2 0 2 0 0 0 0 1 0 0 0]
[1 2 2 0 1 0 0 1 0 1 1 1 0 1 0 0 2 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0]
[0 0 2 1 0 0 0 0 1 0 0 2 2 0 0 0 0 2 0 0 1 0 0 0]
[1 0 2 0 0 2 1 0 1 2 0 2 0 0 2 2 0 2 0 0 0 0 0 1]
[1 0 2 0 0 2 0 1 1 1 0 1 0 0 1 0 2 1 0 0 0 0 1 0]
[1 2 1 1 1 2 1 0 1 1 1 2 1 1 1 1 0 1 1 0 0 0 0 0]
[1 2 2 0 1 2 0 1 1 2 2 2 0 2 2 0 1 2 0 1 0 0 0 0]
[1 1 2 2 2 1 2 2 2 2 1 2 1 1 1 1 2 1 0 0 0 0 0 1]
[2 0 1 0 2 1 2 2 2 2 0 2 0 2 2 2 1 2 0 0 0 0 1 0]


In [23]:
matrix.identity(6) == modular_fourier_transform(2,3)^4

True

In [24]:
order_finite_field(mft)

488488

In [25]:
order_via_lcm(3,5)

6005774712405552325836931245600260388543060466051616552

In [26]:
mft^488488 == matrix.identity(24)

True

In [27]:
SymmetricGroupAlgebra(GF(7),3).dft()

[1 1 1 1 1 1]
[1 4 6 3 3 4]
[0 6 0 6 1 1]
[0 1 0 6 1 6]
[1 3 1 3 3 3]
[1 6 6 1 1 6]

In [28]:
#note: this matrix has (almost) only has 1's and -1's
print(modular_fourier_transform(11,4).str())

[ 1  1  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0  0  0  1]
[ 1  0  1  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0  0 10]
[ 1  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0 10]
[ 1  0  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  1]
[ 1  0  0  0  0  1  0  0  0  0 10  0  0 10  0  0  0  0  1  0  0  0  0  1]
[ 1  1 10 10  1  1  0  0  0  0  0 10 10  0 10 10 10 10 10  0  0  0  0 10]
[ 1  0  0  0  0  0  1  0  0  0  0  1  0  0  0  0  0  0  0  1  0  0  0 10]
[ 1 10  1  0  0  0  1  0  0  0  1  0  0  0 10 10  0  0  0 10  0  0  0  1]
[ 1  0  0  0  0  0  0  1  0  0 10  0  0 10  0  0  0  0  0  0  1  0  0  1]
[ 1  0  0  0  0  0  0  0  1  0  0 10 10  0  0  0  0  0  0  0  0  1  0 10]
[ 1  0  0 10  0  1  0  1  0  0  0  0  1  0  0  0 10  0 10  0 10  0  0 10]
[ 1  1 10 10  0  1  0  0  1  0  0  0  0  1  1  1  1  0  1  0  0 10  0  1]
[ 1  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0  0  0  1  1]
[ 1  0  0 10  1  0  0  0  0  1  0  0  

In [29]:
eigs = modular_fourier_transform(3,5,nu=1).eigenvalues(); lcm([eig.multiplicative_order() for eig in eigs])

6005774712405552325836931245600260388543060466051616552

In [30]:
len(set(eigs))

120

In [31]:
#another way to compute the order of the matrix
lcm([eig.multiplicative_order() for eig in eigs])

6005774712405552325836931245600260388543060466051616552

In [32]:
eigs[20].parent().gen(10)

z10

In [33]:
eigs[1].parent().gen(2)^2 == eigs[1]

False

In [34]:
len(set(eigs))

120

In [35]:
char_poly = modular_fourier_transform(3,4,nu=1).charpoly().factor()

In [36]:
char_poly

(x + 1) * (x^2 + 1) * (x^2 + x + 2) * (x^3 + 2*x^2 + 2*x + 2) * (x^6 + x^5 + 2*x^4 + 2*x^2 + 2*x + 2) * (x^10 + 2*x^8 + x^7 + 2*x^6 + x^4 + 2*x + 2)

In [38]:
[item[0].degree() for item in list(char_poly)]

[1, 2, 2, 3, 6, 10]

In [39]:
factor(566)

2 * 283

In [40]:
#getting PARI stack size error
pari.allocatemem(10^12)
pari.stacksize()

PARI stack size set to 1000000000000 bytes, maximum size set to 1000000012288


1000000000000

In [41]:
K.<a> = modular_fourier_transform(3,4,nu=1).charpoly().splitting_field()

In [42]:
K

Finite Field in a of size 3^30

In [43]:
modular_fourier_transform(2,5,nu=1).charpoly().degree()

120

In [44]:
modular_fourier_transform(3,5,nu=1).charpoly() in GF(3)[x]

True

In [59]:
modular_fourier_transform(2,4,nu=1).charpoly()

x^24 + x^16 + x^8 + 1

In [50]:
R.<x> = PolynomialRing(K)

In [45]:
splitting_field(modular_fourier_transform(3,4,nu=1).charpoly(),GF(3),splitting_degs=[])

(Univariate Quotient Polynomial Ring in xbar over Univariate Quotient Polynomial Ring in xbar over Finite Field of size 3 with modulus x^10 + 2*x^8 + x^7 + 2*x^6 + x^4 + 2*x + 2 with modulus x^3 + (2*xbar^8 + 2*xbar^7 + xbar^5 + xbar^4 + 2*xbar^3 + 2*xbar^2)*x^2 + (2*xbar^8 + 2*xbar^7 + xbar^5 + xbar^4 + 2*xbar^3 + 2*xbar^2 + 1)*x + 2*xbar^8 + 2*xbar^7 + xbar^5 + xbar^4 + 2*xbar^3 + 2*xbar^2,
 [10, 3, 1])

In [46]:
splitting_field_degree(modular_fourier_transform(5,6,nu=1).charpoly())

730140